In [3]:
import numpy as np
import pandas as pd


from sklearn import preprocessing
from datetime import datetime

import pickle
import joblib


import os
import sys 

import pyodbc

from datetime  import datetime

# Log Error  

In [4]:
def WriteLogError(error):
        f = open('ml_error.csv', 'a')
        error_str=f'{datetime.now().strftime("%d/%m/%Y %H:%M:%S")}|{repr(error)}\n'
        f.write(error_str)
        f.close()
        print(error_str) 
        raise error

# Check the most critical variables

In [8]:
x_date=datetime(2020,10,20,10,45,0)
#x_date=datetime.now()


isRunByscheduling=False
isSavedDB=True

#CONSTANT VARIABLE
predictionColName="Signal"

strategyName= 'short'  # Short or Long
strategyName=strategyName.title()

onlyLastRecord=True
isFullFeatures=False


if isRunByscheduling :
 n = len(sys.argv) 
 print("Total arguments passed:", n) 

 try:

  if n==4:
    #sys.argv[0]= program file .py
    strategyName= sys.argv[1].title()
    isFullFeatures=bool(sys.argv[2])
    onlyLastRecord=bool(sys.argv[3])
  else  :
    raise Exception("you must specify the strategyName parameter either of long or short") 
 except Exception as error:
    WriteLogError(error) 

print(f"StrategyTrade={strategyName}")
print(f"FullFeatures={isFullFeatures}")
print(f"Predict one item={onlyLastRecord}")

StrategyTrade=Short
FullFeatures=False
Predict one item=True


# Set all Path Files

In [392]:
root_path=os.path.abspath(r'D:\InvestmentSystem\XAlfa\ML-Project\PredictMLScript\PredictML-Script_V2')
script_path=root_path+'\\AB_Script'
datat_path=root_path+'\\NewData'

holiday_path=os.path.abspath(r'{}\ChkHolidayForYear.csv'.format(script_path))
ml_file_path=os.path.abspath(r'{}\NewData'.format(root_path))
model_path=os.path.abspath(r'{}\Models'.format(root_path))
result_path=os.path.abspath(r'{}\PredictionResult'.format(root_path))

selectedFeat_path=os.path.abspath(r'{}\xgb_s50f15m_featSelect.xlsx'.format(root_path))

os.path.exists(selectedFeat_path)


True

In [393]:

if onlyLastRecord:
  #Live Run with last one data
  abfeatures_file=f'{strategyName}_S50M15_ListFeatures.csv'
else:
  #Live Run with last n-bar data
  abfeatures_file=f'BackFill_S50M15_ListFeatures_{strategyName}.csv'

ml_file_path=f'{ml_file_path}\\{abfeatures_file}'
print(ml_file_path)


D:\InvestmentSystem\XAlfa\ML-Project\PredictMLScript\PredictML-Script_V2\NewData\Short_S50M15_ListFeatures.csv


# Set Model File Path and Feturess (if selectFeature)

In [394]:
if strategyName=='Long':
 sheet_name_feat="long_xgb"
 modelfile=f'{model_path}\\long_S50M15_XGB-80x8x01_01-2007t12-2020_selectFeat_b111020_2226.pickle.dat'
else:
 sheet_name_feat="short_xgb"   
 modelfile=f'{model_path}\\short_S50M15_XGB-160x8x005_01-2007t12-2020_selectFeat_b111020_2228.pickle.dat'

trade_symbol=f'{strategyName}_S50IF_V2' 
#os.path.exists(modelfile)

# Function For Check Market Time

In [395]:
def CheckValidPath(path):
        if os.path.exists(path):
            return True
        else:
            raise Exception(f'Invalid path: {path}') 

In [396]:
# 10:01 is before the first triger in the morning  at 10:02  for first bar at 10:00
# 12:26 is after the last trigter in the morning at 12:17 for first bar at 12:15

# 14:31 is before the first triger after noon  at 14:32  for first bar at 14:30
# 16:56 is after the last trigter after noon at 16:47 for first bar at 16:45

def CheckMarketTime(current_time):
  # check time 10:00-17:00
  isTradeTime_morning=current_time.strftime('%H:%M:%S')>='10:01:00' and current_time.strftime('%H:%M:%S')<='12:36:00' 
  isTradeTime_noon=current_time.strftime('%H:%M:%S')>='14:31:00' and current_time.strftime('%H:%M:%S')<='17:06:00' 
  
  return  isTradeTime_morning or isTradeTime_noon
    

In [397]:
def CheckMarketDay(current_time):
   w_no= int(current_time.strftime('%w'))
   if w_no>=1 and w_no<=5 :
      return True
   else :
      return False

In [398]:
def CheckNotHoliday(df_holiday,current_time):

 holiday_today=current_time.strftime('%Y%m%d')

 df_holiday[0]= current_time.strftime('%Y')+df_holiday[0]
 #print(df_holiday)

 if holiday_today not in df_holiday[0].tolist():
  return True
 else:
  return False
    

# Check Trading Time&Date

In [399]:

print("Current time")
print(x_date.strftime('%d/%m/%Y'))
print(x_date.strftime('%H:%M:%S'))
#print(x_today)

print("===========check trade time&day===========")

IsMarketTime=CheckMarketTime(x_date)
print('IsMarkerTime = ',CheckMarketTime(x_date))

IsMarketDay=CheckMarketDay(x_date)
print('IsMarketDay = ',CheckMarketDay(x_date))


try:
 df_holiday =pd.read_csv(holiday_path,header =  None,dtype=object)

 IsNotHoliday=CheckNotHoliday(df_holiday ,x_date)
 print('IsNotholiday = ',IsNotHoliday)


 IsTradable=IsMarketTime and  IsMarketDay and  IsNotHoliday
 if not IsTradable:
   sys.exit(("It is not trading time")) 


 print("#######################################################")
 print("Let 's go ,it is time to make the large amount of profit")

except Exception as error:
 WriteLogError(error) 
    

Current time
20/10/2020
10:45:00
===========check trade time&day===========
IsMarkerTime =  True
IsMarketDay =  True
IsNotholiday =  True
#######################################################
Let 's go ,it is time to make the large amount of profit


# Check all Path Files

In [400]:
predict_time=datetime.now().strftime('%d%m%y-%H%M')
prediction_file=f'{strategyName}_{predictionColName}_PredictResult_{predict_time}.csv'


path_prediction=f'{result_path}\\{prediction_file}'

print('model:' ,modelfile)
print('new data:',ml_file_path)
print('holiday: ',holiday_path)
print('result: ',path_prediction)

                                                                              
                                                                                    

model: D:\InvestmentSystem\XAlfa\ML-Project\PredictMLScript\PredictML-Script_V2\Models\short_S50M15_XGB-160x8x005_01-2007t12-2020_selectFeat_b111020_2228.pickle.dat
new data: D:\InvestmentSystem\XAlfa\ML-Project\PredictMLScript\PredictML-Script_V2\NewData\Short_S50M15_ListFeatures.csv
holiday:  D:\InvestmentSystem\XAlfa\ML-Project\PredictMLScript\PredictML-Script_V2\AB_Script\ChkHolidayForYear.csv
result:  D:\InvestmentSystem\XAlfa\ML-Project\PredictMLScript\PredictML-Script_V2\PredictionResult\Short_Signal_PredictResult_201020-2322.csv


In [401]:
try:
    if CheckValidPath(modelfile) and CheckValidPath(ml_file_path)  and CheckValidPath(holiday_path)  :
       print("Model File= ",modelfile)
       print("New Data&Features Path= ",ml_file_path)
       print("Holiday Path= ",holiday_path)
     
except Exception as error:
       WriteLogError(error)  

Model File=  D:\InvestmentSystem\XAlfa\ML-Project\PredictMLScript\PredictML-Script_V2\Models\short_S50M15_XGB-160x8x005_01-2007t12-2020_selectFeat_b111020_2228.pickle.dat
New Data&Features Path=  D:\InvestmentSystem\XAlfa\ML-Project\PredictMLScript\PredictML-Script_V2\NewData\Short_S50M15_ListFeatures.csv
Holiday Path=  D:\InvestmentSystem\XAlfa\ML-Project\PredictMLScript\PredictML-Script_V2\AB_Script\ChkHolidayForYear.csv


# 1.Load unseen data file to predict by model

In [402]:

try:
    dataset =pd.read_csv(ml_file_path)
    
    #For one record
    #if onlyLastRecord==True:
     #dataset =dataset.tail(1)
    
    if dataset.shape[0]>0:
        print(f'1-Load unseen data file from {ml_file_path}  succeeded.')
        dataset['Ticker']=trade_symbol
    else:
        raise Exception(f'No feature row  in dataframe from file,please check Amibroker Exploration')    
    
except Exception as error_x:
        WriteLogError(f'{error_x} {ml_file_path}')


print(dataset.info())


1-Load unseen data file from D:\InvestmentSystem\XAlfa\ML-Project\PredictMLScript\PredictML-Script_V2\NewData\Short_S50M15_ListFeatures.csv  succeeded.
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23 entries, 0 to 22
Data columns (total 23 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Ticker                  23 non-null     object 
 1   Date/Time               23 non-null     object 
 2   open                    23 non-null     float64
 3   high                    23 non-null     float64
 4   low                     23 non-null     float64
 5   close                   23 non-null     float64
 6   indy_ma-220             23 non-null     float64
 7   indy_ma-550             23 non-null     float64
 8   indy_hh-220             23 non-null     float64
 9   indy_ll-220             23 non-null     float64
 10  indy_mid-220            23 non-null     float64
 11  indy_hh2-550            23 non-null     float64
 12

In [403]:
#dataset.tail(10)

# 2.Create dataframe succeeded and select features

In [404]:
# create OHLC df
noFeature_col=['Ticker','Date/Time' ,'open','high','low','close']
dfx_ohlc=dataset[noFeature_col]


In [405]:
if  isFullFeatures==True:
    dfx_feature=dataset.drop(columns=noFeature_col)
    print("predict with all features")
else:

    feat_df=pd.read_excel(selectedFeat_path,header=None,engine='openpyxl',sheet_name=sheet_name_feat)
    select_feat=feat_df.loc[:,0].tolist()
    dfx_feature=dataset[select_feat]
    
    print("predict with selective features")

print(dfx_feature.info())
    
print('2-Create dataframe succeeded.')

predict with selective features
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23 entries, 0 to 22
Data columns (total 8 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   indy_ma-550            23 non-null     float64
 1   indy_hh-220            23 non-null     float64
 2   indy_ma-220            23 non-null     float64
 3   cate_CombineTrend      23 non-null     int64  
 4   cate_rannkHL550-ma66   23 non-null     int64  
 5   indy_hh2-550           23 non-null     float64
 6   indy_ll2-550           23 non-null     float64
 7   cate_2trend-550_ma110  23 non-null     int64  
dtypes: float64(5), int64(3)
memory usage: 1.6 KB
None
2-Create dataframe succeeded.


In [406]:
print(dfx_ohlc.tail(5))

            Ticker            Date/Time   open   high    low  close
18  Short_S50IF_V2  20/10/2020 15:45:00  759.9  761.5  758.7  759.3
19  Short_S50IF_V2  20/10/2020 16:00:00  759.4  759.6  757.3  758.6
20  Short_S50IF_V2  20/10/2020 16:15:00  758.7  760.2  758.6  759.8
21  Short_S50IF_V2  20/10/2020 16:30:00  759.7  760.0  758.8  758.9
22  Short_S50IF_V2  20/10/2020 16:45:00  759.1  759.2  758.9  759.0


In [407]:
print(dfx_feature.tail(5))

    indy_ma-550  indy_hh-220  indy_ma-220  cate_CombineTrend  \
18         1.05         1.07         1.03                  2   
19         1.05         1.07         1.03                  2   
20         1.05         1.06         1.03                  2   
21         1.05         1.07         1.03                  2   
22         1.05         1.07         1.03                  2   

    cate_rannkHL550-ma66  indy_hh2-550  indy_ll2-550  cate_2trend-550_ma110  
18                     3          1.10          0.99                      0  
19                     3          1.10          0.99                      0  
20                     3          1.09          0.99                      0  
21                     3          1.10          0.99                      0  
22                     3          1.10          0.99                      0  


# 3.Create numpy array as input for model

In [408]:
X_new=dfx_feature.values
print(X_new[:5,:])
print('3-Create numpy array succeeded.')


[[1.05 1.07 1.04 1.   2.   1.1  1.   0.  ]
 [1.04 1.06 1.03 1.   2.   1.09 0.99 0.  ]
 [1.05 1.06 1.03 1.   2.   1.09 0.99 0.  ]
 [1.05 1.06 1.04 1.   2.   1.09 0.99 0.  ]
 [1.05 1.07 1.04 1.   2.   1.1  1.   0.  ]]
3-Create numpy array succeeded.


# 4.Load model file as strategy for prediction

In [409]:
 try:  
   loaded_model = joblib.load(modelfile)
  
   #loaded_model =  pickle.load(open(modelfile, "rb"))
  
   print(f"4-Loaded model from: {modelfile} succeeded")
 except Exception as error:
   WriteLogError(f'cannot load file {modelfile} {error}')  

4-Loaded model from: D:\InvestmentSystem\XAlfa\ML-Project\PredictMLScript\PredictML-Script_V2\Models\short_S50M15_XGB-160x8x005_01-2007t12-2020_selectFeat_b111020_2228.pickle.dat succeeded


# 5.Predict unseen feature data

In [410]:
yPredited = loaded_model.predict(X_new)
predictions = [round(value) for value in yPredited]

print("Predict Signal : ",predictions)


Predict Signal :  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [411]:
prediction_df=pd.DataFrame(data= {predictionColName: predictions })
print(dfx_ohlc.head(5))
print(prediction_df.head(5))


           Ticker            Date/Time   open   high    low  close
0  Short_S50IF_V2   20/10/2020 9:45:00  760.2  761.2  755.5  758.4
1  Short_S50IF_V2  20/10/2020 10:00:00  758.5  764.8  758.4  763.8
2  Short_S50IF_V2  20/10/2020 10:15:00  764.0  764.3  761.3  761.5
3  Short_S50IF_V2  20/10/2020 10:30:00  761.6  762.4  759.8  760.0
4  Short_S50IF_V2  20/10/2020 10:45:00  760.1  760.2  756.4  757.3
   Signal
0       1
1       1
2       1
3       1
4       1


In [412]:
df_result=pd.concat([dfx_ohlc,prediction_df],axis=1)
print(df_result.head(5))
print('5-Predict new data succeeded.')

            Ticker            Date/Time   open   high    low  close  Signal
0   Short_S50IF_V2   20/10/2020 9:45:00  760.2  761.2  755.5  758.4       1
1   Short_S50IF_V2  20/10/2020 10:00:00  758.5  764.8  758.4  763.8       1
2   Short_S50IF_V2  20/10/2020 10:15:00  764.0  764.3  761.3  761.5       1
3   Short_S50IF_V2  20/10/2020 10:30:00  761.6  762.4  759.8  760.0       1
4   Short_S50IF_V2  20/10/2020 10:45:00  760.1  760.2  756.4  757.3       1
5   Short_S50IF_V2  20/10/2020 11:00:00  757.4  759.9  757.2  759.0       1
6   Short_S50IF_V2  20/10/2020 11:15:00  759.1  759.2  754.6  755.0       1
7   Short_S50IF_V2  20/10/2020 11:30:00  755.1  758.0  753.8  756.9       1
8   Short_S50IF_V2  20/10/2020 11:45:00  757.0  757.2  754.5  754.9       1
9   Short_S50IF_V2  20/10/2020 12:00:00  754.7  756.7  754.3  755.8       1
10  Short_S50IF_V2  20/10/2020 12:15:00  755.9  756.6  755.4  755.8       1
11  Short_S50IF_V2  20/10/2020 12:30:00  755.7  755.9  755.6  755.9       1
12  Short_S5

In [413]:
print("Sample Predict result")
print("=======================================================================")
print(df_result[:5])

Sample Predict result
           Ticker            Date/Time   open   high    low  close  Signal
0  Short_S50IF_V2   20/10/2020 9:45:00  760.2  761.2  755.5  758.4       1
1  Short_S50IF_V2  20/10/2020 10:00:00  758.5  764.8  758.4  763.8       1
2  Short_S50IF_V2  20/10/2020 10:15:00  764.0  764.3  761.3  761.5       1
3  Short_S50IF_V2  20/10/2020 10:30:00  761.6  762.4  759.8  760.0       1
4  Short_S50IF_V2  20/10/2020 10:45:00  760.1  760.2  756.4  757.3       1


# 6.1 Save To CSV

In [384]:
if isSavedDB==False:
 #df_result.drop(columns=['Ticker'],inplace=True)
 df_result.to_csv(path_prediction,index=False)
 print('6-Save predction result to csv succeeded.')


# 6.2 Save To DB (check database server and database name)

In [385]:
try:
 if isSavedDB==True:
  conn = pyodbc.connect('Driver={SQL Server};'
                       'Server=localhost;'
                       'Database=Tfex_AB_ML;'
                       'Trusted_Connection=yes;')

  cursor=conn.cursor()

  for index,row in df_result.iterrows():
      #print(datetime.strptime(row["Date/Time"],'%d/%m/%Y %H:%M:%S'))
      #print(row["Ticker"],row["Date/Time"])
      sqlcmd="INSERT INTO S50F_ML ([SYMBOL],[xDATE],[xOPEN],[xHIGH],[xLOW],[xCLOSE],[xVolume],[Signal],[UpdateTime]) VALUES(?,?,?,?,?,?,?,?,?)"
      cursor.execute(sqlcmd,row["Ticker"],datetime.strptime(row["Date/Time"],'%d/%m/%Y %H:%M:%S'),row["open"],row["high"],row["low"],row["close"],0,row[predictionColName],datetime.now())
      conn.commit()
    
  cursor.commit()
  cursor.close()
  conn.close()
  print('6-Save predction result to database succeeded.')
 #=================================================================
except Exception as error :
  WriteLogError(error)  

6-Save predction result to database succeeded.
